<a href="https://colab.research.google.com/github/jleaoneto/PC_eleicoes_2010_2014_2018/blob/Notebooks/Analise_Eleicoes_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Agregando a coluna NR_RECEITA_TOTAL e CD_ELEITO ao arquivo consulta_cand_2014_RN
import pandas as pd
import numpy as np

#Leitura do arquivo
path = '/content/drive/My Drive/Colab Notebooks/receitas_candidatos_2018_RN.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')
#df.info()

#Conversao para float
df['VR_RECEITA'] = df['VR_RECEITA'].str.replace(',','.').astype('float')

#Agregação das receitas por candidato
df_2 = df.groupby(by=['NR_CPF_CANDIDATO'],as_index=False).agg({'VR_RECEITA' : np.sum})
#df_2.reset_index(inplace=True)
#df_2.rename( columns= {'Valor receita' : 'NR_RECEITA_TOTAL'},inplace=True)
#df_2.info()

#Leitura do arquivo consulta_cand_2018_RN e inclusão da coluna
path_2 = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN.csv'
df_3 = pd.read_csv(path_2, sep=';', encoding='latin1')

print(df_3.query('DS_CARGO == "DEPUTADO ESTADUAL" or DS_CARGO == "DEPUTADO FEDERAL"'))

df_3 = df_3.query('DS_CARGO == "DEPUTADO ESTADUAL" and DS_SITUACAO_CANDIDATURA == "APTO" or DS_CARGO == "DEPUTADO FEDERAL" and DS_SITUACAO_CANDIDATURA == "APTO"')[['ANO_ELEICAO','DS_CARGO','NR_CPF_CANDIDATO','DS_GENERO', 'DS_SIT_TOT_TURNO']]

print(df_3)

df_3 = df_3.merge(df_2, on = 'NR_CPF_CANDIDATO', how='left')


#Função para criar coluna CD_ELEITO (boolean) com base na colunca DS_SIT_TOT_TURNO
def label_eleito (row):
  if row['DS_SIT_TOT_TURNO'] == "ELEITO POR QP" :
    return 1
  elif row['DS_SIT_TOT_TURNO'] == "ELEITO POR MÉDIA" :
    return 1
  else :
    return 0

#Criação da coluna CD_ELEITO
df_3['CD_ELEITO'] = df_3.apply(lambda row : label_eleito(row), axis = 1)

#Preencher as receitas que não estejam atribuídas
df_3.fillna(value={'VR_RECEITA' : 0}, inplace=True)

#Arredondas o valor para duas casas decimais
df_3['VR_RECEITA'] = df_3['VR_RECEITA'].round(decimals=2)

print(df_3)

df_3.to_csv('/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN_com_receita_e_eleito.csv', sep=';', encoding='latin1', index=False)





     DT_GERACAO HH_GERACAO  ...  NR_PROTOCOLO_CANDIDATURA          NR_PROCESSO
0    03/06/2020   15:00:03  ...                        -1  6007340820186200000
1    03/06/2020   15:00:03  ...                        -1  6007359020186200000
2    03/06/2020   15:00:03  ...                        -1  6009333020186200000
3    03/06/2020   15:00:03  ...                        -1  6004543720186200000
4    03/06/2020   15:00:03  ...                        -1  6004899420186200000
..          ...        ...  ...                       ...                  ...
510  03/06/2020   15:00:03  ...                        -1  6003894220186200000
511  03/06/2020   15:00:03  ...                        -1  6008008520186200000
512  03/06/2020   15:00:03  ...                        -1  6008649520186200000
513  03/06/2020   15:00:03  ...                        -1  6005712820186200000
515  03/06/2020   15:00:03  ...                        -1  6006475220186200000

[451 rows x 58 columns]
     ANO_ELEICAO           

In [ ]:
#@title Tabela com relação entre gênero e candidatos
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px 

pd.set_option("display.precision", 2)

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df_1 = df.query('DS_CARGO == "DEPUTADO FEDERAL" or DS_CARGO == "DEPUTADO ESTADUAL"')[['DS_CARGO', 'DS_GENERO']]

df_2 = df_1.groupby(['DS_CARGO','DS_GENERO'])['DS_GENERO'].agg( [ ('FREQ','count') ] )

df_2.reset_index(inplace=True)

fig = px.bar(df_2, barmode='relative', text='FREQ', color_discrete_sequence=('red','blue'), color='DS_GENERO', x='DS_CARGO', y='FREQ')

fig.update_layout(barnorm='percent')

fig.update_yaxes(title='Frequência relativa')

fig.show()


In [ ]:
#@title Tabela com relação entre gênero e eleitos
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px 

pd.set_option("display.precision", 2)

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')


df_grouped = df.query('DS_CARGO == "DEPUTADO ESTADUAL" | DS_CARGO == "DEPUTADO FEDERAL"').groupby(['DS_CARGO','DS_GENERO'], as_index=False)

df_1 = df_grouped['CD_ELEITO'].sum()

fig = px.bar(df_1, y='CD_ELEITO', x='DS_CARGO', color='DS_GENERO', text='CD_ELEITO', color_discrete_sequence=('red','blue'), labels={'DS_GENERO' : 'Gênero', 'DS_CARGO' : 'Cargo', 'CD_ELEITO' : "Eleitos"})

fig.update_layout(barmode='stack')

fig.show()



In [ ]:
#@title Tabela com relação entre gênero e receitas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
#import pandas_profiling 

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df_1 = df.query('DS_CARGO == "DEPUTADO FEDERAL" or DS_CARGO == "DEPUTADO ESTADUAL"')

fig = px.box(df_1, y='VR_RECEITA', boxmode='group', points='outliers' , x = 'DS_CARGO', labels={'CD_ELEITO' : 'Eleito'}, color ='DS_GENERO', color_discrete_sequence=('blue','red'))

fig.show()


In [ ]:
#@title Tabela com relação entre gênero e receitas

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
#import pandas_profiling 

path = '/content/drive/My Drive/Colab Notebooks/consulta_cand_2018_RN_com_receita_e_eleito.csv'
df = pd.read_csv(path, sep=';', encoding='latin1')

df_1 = df.query('DS_CARGO == "DEPUTADO FEDERAL" or DS_CARGO == "DEPUTADO ESTADUAL"')

fig = px.histogram(df_1, x='VR_RECEITA',  barnorm='',  barmode='overlay', facet_col = 'DS_CARGO', color ='DS_GENERO', nbins=6)

fig.show()
